In [45]:
import pandas as pd
import os
import datetime
import json
from functools import reduce

In [32]:
ROOT_DIR = r"C:\dev\backend\tasks\samples\cmots"
asof_date = datetime.date(2022, 12, 7)
DIR_PATH = os.path.join(ROOT_DIR, asof_date.strftime("%d%m%Y"))

DIR_PATH

'C:\\dev\\backend\\tasks\\samples\\cmots\\07122022'

In [33]:
def get_df(dir_path, file_name, asof_date, lst_col, lst_drop_col):
    file_name = os.path.join(dir_path, file_name)
    print(file_name)
    if os.path.isfile(file_name):
        print('Reading the file into df...')
        df = pd.read_csv(file_name, sep ='|', names=lst_col, encoding= 'unicode_escape' )
        df.iloc[:, -1] = df.iloc[:, -1].str.split('<</EOR>>', expand=True)
        # print(df.head())
        df = df.drop(lst_drop_col, axis=1)
    else:
        df = pd.DataFrame()
    return df

In [34]:
# Scheme Master VR mapping
scm_json_file = os.path.join(ROOT_DIR, "scheme_master.json")
scm_mapping = None

# opening JSON file
with open(scm_json_file) as json_file:
    scm_mapping = json.load(json_file)
    
# print the type of scm_mapping variable
# print("Type: ", type(scm_mapping))
# scm_mapping

In [35]:
# get data for scheme master
# 1] dbdmfsch.csv
lst_col = ["MF_COCODE","MF_SCHCODE","SC_CODE","INIT_PRICE","OFFERPRICE","SCHEME","SCHTYPCODE",
                "LAUNC_DATE","CLDATE","INCDATE" ,"SIZE","SIZE_DATE","DIVPAYINT","REDEM_DATE","FUNDCODE",
                "FORM_AVAIL","LIST_UNLST","SECTCODE","AMT_RAISED","OBJECTIVE","MININVT","INC_INVEST",
                "PURREDEM","NAVCALC","TAXBCODE","SPFEATURE","SCHCLCODE","NOTES","FLAG"]
lst_used = ["MF_COCODE","MF_SCHCODE","FUNDCODE","LAUNC_DATE","CLDATE","PURREDEM","INC_INVEST","MININVT"]
lst_drop = [item for item in lst_col if item not in lst_used]

df_dbdmfsch = get_df(DIR_PATH, "dbdmfsch.csv", asof_date, lst_col, lst_drop)
#df_dbdmfsch.head()
#df_dbdmfsch.PURREDEM.unique()


C:\dev\backend\tasks\samples\cmots\07122022\dbdmfsch.csv
Reading the file into df...


In [36]:
# 2] 
# a] dbdindic.csv
lst_col = ["MF_COCODE","MF_SCHCODE","BMCODE","FLAG"]
df_dbdindic = get_df(DIR_PATH, "dbdindic.csv", asof_date, lst_col, [])
df_dbdindic = df_dbdindic[df_dbdindic.FLAG != 'D'] # remove records with FLAG == D
#print(df_dbdindic.head())

# b] dbdbmim.csv
lst_col = ["BMCODE","CO_CODE","SC_CODE","SC_NAME","FLAG"]
lst_used = ["BMCODE","SC_NAME","FLAG"]
lst_drop = [item for item in lst_col if item not in lst_used]
df_dbdbmim = get_df(DIR_PATH, "dbdbmim.csv", asof_date, lst_col, lst_drop)
df_dbdbmim = df_dbdbmim[df_dbdbmim.FLAG != 'D'] # remove records with FLAG == D
#print(df_dbdbmim.head())

df_bmk = pd.merge(df_dbdindic, df_dbdbmim, on="BMCODE", how="left")
df_bmk = df_bmk[["MF_COCODE","MF_SCHCODE","SC_NAME"]]
#df_bmk.head()

C:\dev\backend\tasks\samples\cmots\07122022\dbdindic.csv
Reading the file into df...
C:\dev\backend\tasks\samples\cmots\07122022\dbdbmim.csv
Reading the file into df...


In [37]:
# 3] schclass.csv
lst_col = ["MF_COCODE","MF_SCHCODE","SCH_NAME","MCLASSCODE","MCLASS","SCHEME","SCHTYPCODE",
           "SCH_TYPE","CLASS","CLASS1","PLAN","DIVFRQ","SECTOR","VCLASSCODE","VCLASS","FLAG"]
lst_used = ["MF_COCODE","MF_SCHCODE","CLASS"]
lst_drop = [item for item in lst_col if item not in lst_used]
df_schclass = get_df(DIR_PATH, "schclass.csv", asof_date, lst_col, lst_drop)
#df_schclass = df_schclass[df_schclass.FLAG != 'D'] # remove records with FLAG == D, FLAG is not used as per doc
#df_schclass.head()

C:\dev\backend\tasks\samples\cmots\07122022\schclass.csv
Reading the file into df...


In [38]:
# 4] DBDPLIV2.csv
lst_col = ["MF_COCODE","MF_SCHCODE","PLAN","AVAIL",
           "D1","D2","D3","D4","D5","D6","D7","D8","D9","D10","ANY","OTHERS","FLAG"]
lst_used = ["MF_COCODE","MF_SCHCODE","PLAN","AVAIL","FLAG"]
lst_drop = [item for item in lst_col if item not in lst_used]
df_DBDPLIV2 = get_df(DIR_PATH, "DBDPLIV2.csv", asof_date, lst_col, lst_drop)
df_DBDPLIV2 = df_DBDPLIV2[df_DBDPLIV2.FLAG != 'D'] # remove records with FLAG == D
#df_DBDPLIV2.head()

C:\dev\backend\tasks\samples\cmots\07122022\DBDPLIV2.csv
Reading the file into df...


In [39]:
# 5] mfcat.csv
lst_col = ["MF_COCODE","GRPCODE","MF_SCHCODE","GRPNAME","MF_SCHNAME",
           "PLANNAME","MPLANNAME","OPTIONNAME","MOPTIONNAME"]
lst_used = ["MF_COCODE","MF_SCHCODE","GRPCODE","GRPNAME","MF_SCHNAME",
            "PLANNAME","MPLANNAME","OPTIONNAME","MOPTIONNAME"]
lst_drop = [item for item in lst_col if item not in lst_used]
df_mfcat = get_df(DIR_PATH, "mfcat.csv", asof_date, lst_col, lst_drop)
#df_mfcat.head()

C:\dev\backend\tasks\samples\cmots\07122022\mfcat.csv
Reading the file into df...


In [40]:
# 6] mfisin_all.csv
lst_col = ["MF_SCHCODE","ISIN"]
lst_used = ["MF_SCHCODE","ISIN"]
lst_drop = [item for item in lst_col if item not in lst_used]
df_mfisin_all = get_df(DIR_PATH, "mfisin_all.csv", asof_date, lst_col, lst_drop)
#df_mfisin_all.head()

C:\dev\backend\tasks\samples\cmots\07122022\mfisin_all.csv
Reading the file into df...


In [41]:
# 7] amfimaster.csv
lst_col = ["MF_COCODE","MF_SCHCODE","SRNO","SCH_NAME","AMFICODE","FLAG"]
lst_used = ["MF_COCODE","MF_SCHCODE","AMFICODE","SCH_NAME"]
lst_drop = [item for item in lst_col if item not in lst_used]
df_amfimaster = get_df(DIR_PATH, "amfimaster.csv", asof_date, lst_col, lst_drop)
#df_amfimaster.head()

C:\dev\backend\tasks\samples\cmots\07122022\amfimaster.csv
Reading the file into df...


In [42]:
# 8] catmas_sebi.csv
lst_col = ["MF_SCHCODE","CATCODE","CATNAME","SUBCATCODE","SUBCATNAME"]
lst_used = ["MF_SCHCODE","CATCODE","CATNAME","SUBCATCODE","SUBCATNAME"]
lst_drop = [item for item in lst_col if item not in lst_used]
df_catmas_sebi = get_df(DIR_PATH, "catmas_sebi.csv", asof_date, lst_col, lst_drop)
#df_catmas_sebi.head()

C:\dev\backend\tasks\samples\cmots\07122022\catmas_sebi.csv
Reading the file into df...


In [50]:
# Join all the df above
df_list = [df_dbdmfsch, df_bmk, df_schclass, df_DBDPLIV2, df_mfcat, df_mfisin_all, df_amfimaster, df_catmas_sebi]

df_dbdmfsch = df_dbdmfsch.merge(df_bmk[["MF_SCHCODE","SC_NAME"]], on="MF_SCHCODE", how='left')

df_dbdmfsch = df_dbdmfsch.merge(df_schclass[["MF_SCHCODE","CLASS"]], on="MF_SCHCODE", how='left')

df_dbdmfsch = df_dbdmfsch.merge(df_DBDPLIV2[["MF_SCHCODE","PLAN","AVAIL"]], on="MF_SCHCODE", how='left')

df_dbdmfsch = df_dbdmfsch.merge(df_mfcat[["MF_SCHCODE","GRPNAME","MF_SCHNAME",
                                          "PLANNAME","MPLANNAME","OPTIONNAME","MOPTIONNAME"]], on="MF_SCHCODE", how='left')

df_dbdmfsch = df_dbdmfsch.merge(df_mfisin_all, on="MF_SCHCODE", how='left')

df_dbdmfsch = df_dbdmfsch.merge(df_amfimaster[["MF_SCHCODE","AMFICODE","SCH_NAME"]], on="MF_SCHCODE", how='left')

df_dbdmfsch = df_dbdmfsch.merge(df_catmas_sebi[["MF_SCHCODE","CATNAME","SUBCATNAME"]], on="MF_SCHCODE", how='left')

df_dbdmfsch.to_csv(r"C:\dev\backend\tasks\samples\cmots\Merged.csv")


In [ ]:
dbdfundm = get_df(DIR_PATH, "dbdfundm.csv", target_date, ["FUNDCODE", "FUND_MGR", "FLAG"], [])
dbdprof = get_df( DIR_PATH, "dbdprof.csv", target_date, ["FUNDCODE", "QUAL", "DESIG", "EXPERIENCE", "REMARKS", "FLAG"], [])
mffundmgr = get_df( DIR_PATH, "mffundmgr.csv", target_date, ["mf_schcode", "fundcode", "fund_mgr"], [])

fund_manager= pd.merge(dbdfundm.drop("FLAG", axis=1), dbdprof.drop("FLAG", axis=1), how="outer", left_on="FUNDCODE", right_on="FUNDCODE")

In [ ]:
dbdbmim = get_df( DIR_PATH, "dbdbmim.csv", target_date, ["BMCODE", "CO_CODE", "SC_CODE", "SC_NAME", "FLAG"], ["SC_CODE", "FLAG"])
dbdindic = get_df( DIR_PATH, "dbdindic.csv", target_date, ["MF_COCODE", "MF_SCHCODE", "BMCODE", "FLAG"], ["FLAG"])
bm = pd.merge(dbdindic, dbdbmim, how="left", on="BMCODE")

catmas_sebi = get_df( DIR_PATH, "catmas_sebi.csv", target_date, ["mf_schcode", "catcode", "catname", "subcatcode", "subcatname"], [])
mfriskometer = get_df( DIR_PATH, "mfriskometer.csv", target_date, ["mf_schcode", "riskometercode", "riskometervalue"], [])
mfisin_all = get_df( DIR_PATH, "mfisin_all.csv", target_date, ["mf_schcode", "ISIN"], [])
mfdirindir = get_df( DIR_PATH, "mfdirindir.csv", target_date, ["mf_schcode", "investment"], [])

schclass = get_df( DIR_PATH, "schclass.csv", target_date, 
                  ["MF_COCODE", "MF_SCHCODE", "SCH_NAME", "MCLASSCODE", "MCLASS", "SCHEME", "SCHTYPCODE", "SCH_TYPE", "CLASS", "CLASS1", 
                   "PLAN", "DIVFRQ", "SECTOR", "VCLASSCODE", "VCLASS", "flag"], 
                  ["MCLASSCODE", "MCLASS", "CLASS1", "SECTOR", "flag"]
                 )
amfimaster = get_df(DIR_PATH, "amfimaster.csv", target_date, 
                    ["MF_COCODE", "MF_SCHCODE", "SRNO", "SCH_NAME", "AMFICODE", "FLAG"], 
                    ["SRNO", "FLAG"])
mfcat = get_df( DIR_PATH, "mfcat.csv", target_date, 
               ["MF_COCODE", "GRPCODE", "MF_SCHCODE", "GRPNAME", "MF_SCHNAME", "PLANNAME", "MPLANNAME", "OPTIONNAME", "MOPTIONNAME"],
               [])
d = pd.merge(catmas_sebi, mfriskometer, how="left", on=["mf_schcode"])
d = pd.merge(d, mfisin_all, how="left", on=["mf_schcode"])
d = pd.merge(d, mfdirindir, how="left", on=["mf_schcode"])

d1 = pd.merge(schclass, amfimaster, how="left", on=["MF_COCODE", "MF_SCHCODE", "SCH_NAME"])
mfcat = mfcat.drop(["MF_SCHNAME"], axis=1)

d1 = pd.merge(d1, mfcat, how="left", on=["MF_COCODE", "MF_SCHCODE"])

plans = pd.merge(d1, d, how="left", left_on="MF_SCHCODE",right_on="mf_schcode")
plans = plans.drop(["mf_schcode"], axis=1)
plans = pd.merge(plans, bm, how="left", on=["MF_COCODE", "MF_SCHCODE"])

In [ ]:
dbdprice = get_df( DIR_PATH, "dbdprice.csv", target_date, 
                  ["MF_COCODE", "MF_SCHCODE", "SCHTYPCODE", "NAVDATE", "NAVRS", "ADJNAVRS", "NAVREPRICE", "NAVSAPRICE", "NAVLSPRICE", "RC", "SC", "FLAG"],
                  ["NAVLSPRICE", "RC", "SC"])

dbdprice_n = get_df( DIR_PATH, "dbdprice_n.csv", target_date, 
                  ["MF_COCODE", "MF_SCHCODE", "SCHTYPCODE", "NAVDATE", "NAVRS", "ADJNAVRS", "NAVREPRICE", "NAVSAPRICE", "NAVLSPRICE", "RC", "SC", "FLAG"],
                  ["NAVLSPRICE", "RC", "SC"])

nav = dbdprice
nav_n = dbdprice_n

In [ ]:
dbdpfoli = get_df( DIR_PATH, "dbdpfoli.csv", target_date, 
                  ["MF_COCODE", "MF_SCHCODE", "SCHTYPCODE", "INVDATE", "TYPECODE", "STYPECODE", "DEBTYPCODE", "RATINGCODE", "RATING", "PERIOD", 
                   "PEROFCODE", "TOTNAV", "MKTVALUE", "REPSECTOR", "INV_NO", "CO_CODE", "CO_NAME", "NO_SHARES", "PERC_HOLD", "INTPER", "EDATE",
                   "GROUP", "COMCODE", "COMNAME", "TCODE", "TNAME", "TOPPF", "FLAG"], 
                  ["STYPECODE", "DEBTYPCODE", "PEROFCODE", 
                   "EDATE", "GROUP", "TOPPF"])

dbdnptm = get_df( DIR_PATH, "dbdnptm.csv", target_date, 
                 ["COMCODE", "COMNAME", "TYPECODE", "TYPE", "SUBCODE", "SUBTYPNAME", "DEBTYPCODE", "DEBTYPE", 
                  "LEV1", "LEV2", "LEV3", "OLD1", "OLD2", "OLD3"],
                 ["LEV1", "LEV2", "LEV3", "OLD1", "OLD2", "OLD3"])

dbdderiv = get_df( DIR_PATH, "dbdderiv.csv", target_date, 
                  ["MF_COCODE", "MF_SCHCODE", "SCHTYPCODE", "INVDATE", "TYPECODE", "STYPECODE", "DEBTYPCODE", "RATINGCODE", "RATING", "PERIOD", 
                   "PEROFCODE", "TOTNAV", "MKTVALUE", "REPSECTOR", "INV_NO", "CO_CODE", "CO_NAME", "NO_SHARES", "PERC_HOLD", "INTPER", 
                   "COMCODE", "COMNAME", "TCODE", "TNAME", "FLAG"], 
                  ["STYPECODE", "DEBTYPCODE", "RATINGCODE", "RATING",
                   "PEROFCODE", "INTPER", "TCODE", "TNAME", "FLAG"])

# Derivatives are not used as of now
holdings = pd.merge(dbdpfoli, dbdnptm, how="left", on=["TYPECODE", "COMCODE"])

In [ ]:
# cardv = get_df( DIR_PATH, "cardv.csv", target_date, 
#                [ "DATE", "SCHCLCODE", "SCHCLASS", "1DAYMAX", "1DAYMIN", "1DAYAVG", "WEEKMAX", "WEEKMIN", "WEEKAVG", "1MMAX", "1MMIN", "1MAVG", 
#                 "3MMAX", "3MMIN", "3MAVG", "6MMAX", "6MIN", "6MAVG", "1YRMAX", "1YRMIN", "1YRAVG", "3YRMAX", "3YRMIN", "3YRAVG", 
#                 "5YRMAX", "5YRMIN", "5YRAVG", "INCMAX", "INCMIN", "INCAVG"],
#                [ "1DAYMAX", "1DAYMIN", "WEEKMAX", "WEEKMIN", "1MMAX", "1MMIN", "3MMAX", "3MMIN", "6MMAX", "6MIN", "1YRMAX", "1YRMIN", 
#                 "3YRMAX", "3YRMIN", "5YRMAX", "5YRMIN", "INCMAX", "INCMIN"
#                ]
#               )

# catmas = get_df(DIR_PATH, "catmas.csv", target_date, ["VCLASSCODE", "VCLASS", "AssetType1", "AssetType2"], [])

# dbddiv = get_df( DIR_PATH, "dbddiv.csv", target_date, [
#     "MF_COCODE", "MF_SCHCODE", "SCHTYPCODE", "DIVTYPCODE", "DIVTYPE", "NAVREFDATE", "BONUSRATE1", "BONUSRATE2", "DIVDATE", "DIVPER", 
#     "CDIVPER", "NCDIVPER", "EXNAV", "DIVAMT", "DIVACPER", "DIVACPDATE", "DIVACP", "DIVACFDATE", "DIVPERPU", "RECORDDATE", "REMARK", 
#     "INTDATE", "FV", "FLAG"],
#                ["EXNAV", "DIVAMT", "DIVACPDATE", "DIVACFDATE", "INTDATE"]
#                )

# dbddivtm = get_df( DIR_PATH, "dbddivtm.csv", target_date, [
#     "DIVTYPCODE", "DIVTYPE", "FLAG"],
#                   []
#                  )

# dbdaum = get_df( DIR_PATH, "dbdaum.csv", target_date, [
#     "MF_COCODE", "MF_SCHCODE", "AMCDATE", "AMCSIZE", "FLAG"
# ], [])

# dbdexpr = get_df( DIR_PATH, "dbdexpr.csv", target_date, [
#     "MF_COCODE", "MF_SCHCODE", "ERDATE", "EXPRATIO", "EXPAMT", "UNITCAP", "NOOFUNITS", "FLAG"
# ], [])

# latnav = get_df( DIR_PATH, "latnav.csv", target_date, 
#                 ["MF_SCHCODE","NAVDATE","NAVRS","NAVREPRICE","NAVSAPRICE","NAVLSPRICE","CLDATE","SCH_NAME","LNAME","CO_CODE"], 
#                 ["NAVLSPRICE","CLDATE","LNAME"])

# latnav_n = get_df( DIR_PATH, "latnav_n.csv", target_date, 
#                 ["MF_SCHCODE","NAVDATE","NAVRS","NAVREPRICE","NAVSAPRICE","NAVLSPRICE","CLDATE","SCH_NAME","LNAME","CO_CODE"], 
#                 ["NAVLSPRICE","CLDATE","LNAME"])

In [ ]:
dbdload2 = get_df( DIR_PATH, "dbdload2.csv", target_date, [
    "MF_COCODE", "MF_SCHCODE", "LOADDATE", "LOADTYPE", "LOAD_NO", "FRDATE", "TODATE", "FRAMT", "UPTOAMT", "MINPERIOD", "MAXPERIOD", 
    "MIN", "MAX", "ENTRYLOAD", "EXITLOAD", "REMARKS", "FLAG"],
                 [])

dbdmfsch = get_df( DIR_PATH, "dbdmfsch.csv", target_date, 
                  ["MF_COCODE", "MF_SCHCODE", "SC_CODE", "INIT_PRICE", "OFFERPRICE", "SCHEME", "SCHTYPCODE", "LAUNC_DATE", "CLDATE", "INCDATE", 
                   "SIZE", "SIZE_DATE", "DIVPAYINT", "REDEM_DATE", "FUNDCODE", "FORM_AVAIL", "LIST_UNLST", "SECTCODE", "AMT_RAISED", "OBJECTIVE", 
                   "MININVT", "INC_INVEST", "PURREDEM", "NAVCALC", "TAXBCODE", "SPFEATURE", "SCHCLCODE", "NOTES", "FLAG"], 
                  ["SC_CODE", "FORM_AVAIL", "LIST_UNLST", "SECTCODE", "AMT_RAISED", "TAXBCODE", "SPFEATURE", "SCHCLCODE"]
                 )

returns = get_df( DIR_PATH, "return.csv", target_date, 
                 ["MF_COCODE", "MF_SCHCODE", "SCH_NAME", "DATE", "TODATE", "TONAV", "TONAVP", "PNAVFROM", "PNAV", "PNAVP", "PRET", 
                  "1WEEKFRM", "1WEEKNAV", "1WEEKNAVP", "1WEEK", "1MTHFRM", "1MTHNAV", "1MTHNAVP", "1MONTH", "3MTHFRM", "3MTHNAV", "3MTHNAVP", "3MONTH", 
                  "6MNTFROM", "6MNTNAV", "6MNTNAVP", "6MONTH", "1YRFROM", "1YRNAV", "1YRNAVP", "1YEAR", "3YRFROM", "3YRNAV", "3YRNAVP", "3YEAR", 
                  "5YRFROM", "5YRNAV", "5YRNAVP", "5YEAR", "INCFROM", "INCNAV", "INCEPTION", 
                  "FUNDSIZE", "SCHCLCODE", "CLASSNAME", "TYPECODE", "AMC_CODE", "AMCNAME", "NAVDATE", "NAVRS", "LAUNCHDT"], 
                 ["TONAVP", "PNAVFROM", "PNAV", "PNAVP", "PRET", "1WEEKNAVP", "1MTHNAVP", "3MTHNAVP", "6MNTNAVP", "1YRNAVP", "3YRNAVP",
                  "5YRNAVP", "NAVDATE", "NAVRS"]
                )

cardsebi = get_df( DIR_PATH, "cardsebi.csv", target_date, 
                  ["DATE", "subcat_code", "subcate_name", "1DAYMAX", "1DAYMIN", "1DAYAVG", "WEEKMAX", "WEEKMIN", "WEEKAVG", "1MMAX", "1MMIN", "1MAVG",
                   "3MMAX", "3MMIN", "3MAVG", "6MMAX", "6MIN", "6MAVG", "1YRMAX", "1YRMIN", "1YRAVG", "3YRMAX", "3YRMIN", "3YRAVG", 
                   "5YRMAX", "5YRMIN", "5YRAVG", "INCMAX", "INCMIN", "INCAVG"], 
                  ["1DAYMAX", "1DAYMIN", "WEEKMAX", "WEEKMIN", "1MMAX", "1MMIN", "3MMAX", "3MMIN", "6MMAX", "6MIN", "1YRMAX", "1YRMIN",
                   "3YRMAX", "3YRMIN", "5YRMAX", "5YRMIN", "INCMAX", "INCMIN"])

dbdavgm = get_df( DIR_PATH, "dbdavgm.csv", target_date, [
     "MF_COCODE", "MF_SCHCODE", "SCHTYPCODE", "INVDATE", "AVG_MAT1", "AVG_MAT2", "YTM", "AVG_PF1", "AVG_PF2", "FLAG"
], [])

dbdmprof = get_df( DIR_PATH, "dbdmprof.csv", target_date, [
     "MF_COCODE", "MF_SCHCODE", "INVDATE", "TYPECODE", "MAT_NO", "P1", "P2", "P3", "P4", "PERC_HOLD", "MKTVALUE", "FLAG"
], [])

dbdptm = get_df( DIR_PATH, "dbdptm.csv", target_date, [
    "TYPECODE", "TYPE", "FLAG"
], [])

dbdtor = get_df( DIR_PATH, "dbdtor.csv", target_date, [
    "MF_COCODE", "MF_SCHCODE", "BMCODE", "FLAG"
], [])

dbdtor = get_df( DIR_PATH, "dbdtor.csv", target_date, [
    "mf_cocode", "mf_schcode", "torDate", "torRatio", "torTimes", "flag"
], [])

DBDPLIV1 = get_df( DIR_PATH, "DBDPLIV1.csv", target_date, [
    "MF_COCODE", "MF_SCHCODE", "PLAN", "LOAD", "REMARK", "COTPURCH", "COTREDEM", "FLAG"
], [])

DBDPLIV2 = get_df( DIR_PATH, "DBDPLIV2.csv", target_date, [
    "MF_COCODE", "MF_SCHCODE", "PLAN", "AVAIL", "D1", "D2", "D3", "D4", "D5", "D6", "D7", "D8", "D9", "D10", "ANY", "OTHERS", "FLAG"
], [])

DBDPLIV3 = get_df( DIR_PATH, "DBDPLIV3.csv", target_date, [
    "MF_COCODE", "MF_SCHCODE", "PLAN", "FREQ", "SLNO", "MINAMT", "MINUNIT", "MULTAMT", "MULTUNIT", "INSTLMNT", "DURATION", "FLAG"
], [])

amcsiz = get_df( DIR_PATH, "amcsiz.csv", target_date, [
    "MF_COCODE", "MF_SCHCODE", "INVDATE", "TOTNAV", "FLAG"
], [])

DBDYTDER = get_df( DIR_PATH, "DBDYTDER.csv", target_date, [
    "MF_COCODE", "MF_SCHCODE", "YTDERDATE", "YTDER", "ER", "WAER", "FLAG"
], [])

In [ ]:
plans

In [ ]:
cardsebi

In [ ]:
schclass

In [ ]:
mfcat